## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os, random, codecs, json, pickle
seed = 99
random.seed(seed)
np.random.seed(seed)
from collections import Counter

In [2]:
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.test.utils 
import tqdm
import itertools

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from pyvis.network import Network

In [4]:
from imp import reload
import pyLDAvis
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_4344/2111997776.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Import and parse Data

In [6]:
#read the dataframe
lavrov = pd.read_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_lavrov/lavrov_5_col.pkl')

In [15]:
lavrov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5212
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   speaker  5208 non-null   object        
 1   date     5208 non-null   datetime64[ns]
 2   url      5208 non-null   object        
 3   title    5208 non-null   object        
 4   text     5208 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 244.1+ KB


In [17]:
lavrov_clean = pd.read_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_lavrov/lavrov_clean.pkl')

In [18]:
lavrov_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5212
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   speaker     5208 non-null   object        
 1   date        5208 non-null   datetime64[ns]
 2   url         5208 non-null   object        
 3   title       5208 non-null   object        
 4   text        5208 non-null   object        
 5   word_count  5208 non-null   int64         
 6   lemma       5208 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 325.5+ KB


In [23]:
lavrov_clean.rename(columns={'text':'text_clean'}, inplace=True)

In [24]:
col_to_add = lavrov_clean['text_clean']

In [25]:
lavrov_concatenated = pd.concat([lavrov, col_to_add], axis=1)
lavrov_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208 entries, 0 to 5212
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   speaker     5208 non-null   object        
 1   date        5208 non-null   datetime64[ns]
 2   url         5208 non-null   object        
 3   title       5208 non-null   object        
 4   text        5208 non-null   object        
 5   text_clean  5208 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 284.8+ KB


In [26]:
lavrov_concatenated = lavrov_concatenated[['date','url','title','speaker','text','text_clean']]

In [27]:
lavrov_concatenated.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/lavrov_concatenated.pkl')

In [28]:
lavrov_concatenated = pd.read_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/lavrov_concatenated.pkl')

In [8]:
output_dir = '/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia'
names = ['somi_concatenated','news_concatenated', 'messages_to_federal_assembly','interviews_concatenated']

for name in names:
    file_path = os.path.join(output_dir, f"{name}.pkl")
    with open(file_path, "rb") as file:
        df = pd.read_pickle(file)
        globals()[name] = df

In [9]:
somi_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        691 non-null    object
 1   url         691 non-null    object
 2   title       691 non-null    object
 3   speaker     691 non-null    object
 4   text        691 non-null    object
 5   text_clean  691 non-null    object
dtypes: object(6)
memory usage: 32.5+ KB


In [10]:
interviews_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        114 non-null    object
 1   url         114 non-null    object
 2   title       114 non-null    object
 3   speaker     114 non-null    object
 4   text        114 non-null    object
 5   text_clean  114 non-null    object
dtypes: object(6)
memory usage: 5.5+ KB


In [12]:
messages_to_federal_assembly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        14 non-null     object
 1   URL         14 non-null     object
 2   title       14 non-null     object
 3   speaker     14 non-null     object
 4   text        14 non-null     object
 5   text_clean  14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [44]:
messages_to_federal_assembly.rename(columns={'URL':'url'}, inplace=True)

In [14]:
news_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         754 non-null    object
 1   URL          754 non-null    object
 2   description  754 non-null    object
 3   speaker      754 non-null    object
 4   text         754 non-null    object
 5   text_clean   754 non-null    object
dtypes: object(6)
memory usage: 35.5+ KB


In [45]:
news_concatenated.rename(columns={'URL':'url'}, inplace=True)

In [53]:
news_concatenated.rename(columns={'description':'title'}, inplace=True)

In [54]:
# for each dataset, add a col word_count
#combine the dfs in a list
dfs = [somi_concatenated, messages_to_federal_assembly, interviews_concatenated, news_concatenated,lavrov_concatenated]
# iterate over each dataframe and extract the word count for the 'text_clean' column and cut row where word_count == 0
for df in dfs:
    df['word_count'] = df['text_clean'].apply(lambda x: len(x.split()))
    df.drop(df[df['word_count'] == 0].index, inplace=True)

In [55]:
merged_df = pd.concat([somi_concatenated, messages_to_federal_assembly, interviews_concatenated, news_concatenated,lavrov_concatenated], axis=0)

In [56]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6779 entries, 0 to 5212
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        6779 non-null   object
 1   url         6779 non-null   object
 2   title       6779 non-null   object
 3   speaker     6779 non-null   object
 4   text        6779 non-null   object
 5   text_clean  6779 non-null   object
 6   word_count  6779 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 423.7+ KB


In [57]:
#save as pickle
merged_df.to_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/merged_df.pkl')

## Text preprocessing for word embedding

col text_clean is based on this function:

def clean(text):
    # replace actual newline characters \n with a space
    text = re.sub(r'\n', ' ', text)
    #replace Д.Медведев:|В.Путин: with a white space
    text = re.sub(r'(Д\.Медведев:|В\.Путин:)', ' ', text)
    # replace everything non-alphanumeric with a space
    text = re.sub(r'\W+', ' ', text)
    # replace two or more dots with one
    text = re.sub(r'\.{2,}', ' ', text)
    # replace sequences of white spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # convert text to lowercase
    text = text.lower()
    # replace \xa0 with a white space
    text = re.sub(r'\xa0', ' ', text)
    return text.strip()

In [3]:
df = pd.read_pickle('/Users/adrianacuppuleri/Desktop/GITHUB ADRIANA/Illiberal_discourse/data/corpus_adriana/corpus_president_of_russia/merged_df.pkl')

chunk the df into 4 time frame according to presidential mandate

grid search with the best word2vec model

loop on each time frame and make bootstrap sampling and word2vec

then run again the loop but this time just for the interested word

get the cosine similarity between the matrix of entire corpus and the matrix of the single word

get the mean of the bootstrap sample for vectors of sigle word and for vectors of all words from the corpus